<a href="https://colab.research.google.com/github/ecsquare/Covid-19/blob/master/Short_text_based_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Topic modeling for short texts is an increasingly impor-tant task due to the prevalence of short texts on the Web.Compared to normal documents, short texts lack of wordfrequency and context information, causing severe sparsityproblems for conventional topic models


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/vegan_tweets_2020-05-/

/content/gdrive/My Drive/Colab Notebooks/vegan_tweets_2020-05-


In [4]:
ls

 vegan_2020-05-01.json      vegan_2020-05-04.json  'vegan_2020-05-07 2.json'
'vegan_2020-05-02 2.json'   vegan_2020-05-05.json
 vegan_2020-05-03.json      vegan_2020-05-06.json


preparing Dataset


In [4]:
import pandas as pd
df1 = pd.read_json (r'vegan_2020-05-01.json',lines=True)
df2 = pd.read_json(r'vegan_2020-05-02 2.json', lines=True)
df3 = pd.read_json(r'vegan_2020-05-03.json', lines=True)
df4 = pd.read_json(r'vegan_2020-05-04.json', lines=True)
df5 = pd.read_json(r'vegan_2020-05-05.json', lines=True)
df6 = pd.read_json(r'vegan_2020-05-06.json', lines=True)
df7 = pd.read_json(r'vegan_2020-05-07 2.json', lines=True)

frames = [df1, df2, df3,df4,df5,df6,df7]
df=pd.concat(frames)
df=df[['id','tweet','hashtags']]
df.columns
df.head(10)

,id,tweet,hashtags
0,1256372793752653825,how bout y’all compare shit that actually make...,[]
1,1256372790497644545,( http://HappyCow.net is an excellent search...,[]
2,1256372760638566401,Whew RT @_underhereye: Brussels Sprouts cooked...,[]
3,1256372759619350528,vegan labouyi (plantiain porridge). one of my ...,[]
4,1256372716141232129,Sounds a bit like a veterinarian vegan sort of...,[]
5,1256372705940705280,"no girl has all three: synesthesia, internal m...",[]
6,1256372679784960006,I definitely need to do my hair in NYC again soon,[]
7,1256372662210879489,how u goin vegan but still beefin w me again?!...,[#newprofilepic]
8,1256372646117117952,Happy Friday! I had some @traderjoes herb pizz...,[#vegan]
9,1256372631130894336,Which do you prefer? Gummies or capsules? ⭐️⭐️...,"[#hairtamin, #vegan, #vegeterian, #hairvitamins]"


Data cleaning

In [5]:
import string 
import re

def clean_tweets(text):

    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
def cleanhashtag(text):
    text = clean_tweets(text)
    text= " ".join(text.split('#'))
    return text

    
df['tweet'] = df['tweet'].apply(lambda x: clean_tweets(x))
df['hashtags']=  df['hashtags'].apply(lambda x: cleanhashtag(x))
df.head(10)

,id,tweet,hashtags
0,1256372793752653825,how bout y’all compare shit that actually make...,
1,1256372790497644545,httpHappyCownet is an excellent search tool...,
2,1256372760638566401,Whew RT underhereye Brussels Sprouts cooked in...,
3,1256372759619350528,vegan labouyi plantiain porridge one of my fav...,
4,1256372716141232129,Sounds a bit like a veterinarian vegan sort of...,
5,1256372705940705280,no girl has all three synesthesia internal mon...,
6,1256372679784960006,I definitely need to do my hair in NYC again soon,
7,1256372662210879489,how u goin vegan but still beefin w me again N...,newprofilepic
8,1256372646117117952,Happy Friday I had some traderjoes herb pizza ...,vegan
9,1256372631130894336,Which do you prefer Gummies or capsules ⭐️⭐️ h...,hairtamin vegan vegeterian hairvitamins


In [8]:
df["Tweet Length"]= df["tweet"].str.len() 
df.head(10)

,id,tweet,hashtags,Tweet Length
0,1256372793752653825,how bout y’all compare shit that actually make...,,121
1,1256372790497644545,httpHappyCownet is an excellent search tool...,,250
2,1256372760638566401,Whew RT underhereye Brussels Sprouts cooked in...,,110
3,1256372759619350528,vegan labouyi plantiain porridge one of my fav...,,106
4,1256372716141232129,Sounds a bit like a veterinarian vegan sort of...,,52
5,1256372705940705280,no girl has all three synesthesia internal mon...,,58
6,1256372679784960006,I definitely need to do my hair in NYC again soon,,49
7,1256372662210879489,how u goin vegan but still beefin w me again N...,newprofilepic,79
8,1256372646117117952,Happy Friday I had some traderjoes herb pizza ...,vegan,286
9,1256372631130894336,Which do you prefer Gummies or capsules ⭐️⭐️ h...,hairtamin vegan vegeterian hairvitamins,108


In [9]:
print("mean=", df['Tweet Length'].mean(), "max=", df['Tweet Length'].max(), "min=",df['Tweet Length'].min())

mean= 128.78752052545155 max= 477 min= 4


In [10]:
df.hashtags.value_counts()

                                                                                                                                   10148
 vegan                                                                                                                               212
 shopmycloset                                                                                                                         26
 plantbasedparty                                                                                                                      15
 recipes food cooking delicious cook recipe                                                                                           13
                                                                                                                                   ...  
 conference authors brendadavis annamariaclement brianclement joelfuhrman foods plantbasednutrition vegan vegetarian wholefoods        1
 vegan itseasybeingvegan poweredbyplants 


Vectorization with sklearn.



In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cvec = CountVectorizer(stop_words='english', min_df=1, max_df=.5, ngram_range=(1,2))
cvec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

BTM trial

In [0]:
import json

data = [json.loads(line)['tweet'] for line in open('vegan_2020-05-01.json', 'r')]



In [0]:
data = [clean_tweets(d) for d in data]

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(data).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
X[:50]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
!pip install biterm

In [0]:
!pip install pyLDAvis

In [0]:
import numpy as np 
from biterm.utility import vec_to_biterms

vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X[:50])

In [0]:
biterms

In [0]:
from biterm.btm import oBTM

btm = oBTM(num_topics=20, V=vocab)
  



In [0]:
topics = btm.fit_transform(biterms, iterations=20)

In [104]:
print("\n\n Visualize Topics ..")

vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X[:50], axis=1), vocab, np.sum(X[:50], axis=0))
pyLDAvis.save_html(vis, 'online_btm.html')




 Visualize Topics ..


In [103]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.212211 -0.009554       1        1  7.958381
19     0.025256  0.088885       2        1  7.770740
14    -0.066620  0.092044       3        1  7.627874
17    -0.253157 -0.107492       4        1  7.594801
5      0.000920  0.071742       5        1  7.441028
6      0.056221  0.100796       6        1  6.724945
16     0.047726  0.012452       7        1  6.509012
1     -0.094369  0.053475       8        1  6.483740
11    -0.048639  0.131888       9        1  5.880598
18     0.051351  0.010280      10        1  5.775315
2     -0.245478 -0.148942      11        1  5.265237
9      0.040134 -0.092839      12        1  4.429906
7     -0.015471  0.258972      13        1  4.307511
4      0.096002  0.016331      14        1  3.887384
8      0.095641 -0.000963      15        1  3.705980
12     0.027533 -0.078256      16        1  3.694414
10     0.044995

In [0]:
! pip install pyLDAvis
